In [1]:
import pandas as pd
from datetime import datetime
from slugify import slugify

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [3]:
now = datetime.now()
dt_string = now.strftime("%Y-%m-%d %H:%M:%S")

In [4]:
df = pd.read_csv('csv/products.csv')
priceDf = pd.read_csv('out/price.csv')
usersDf = pd.read_csv('csv/users.csv')
users = pd.read_csv('out/users.csv')
locDf = pd.read_csv('out/localities.csv')
prodDf = pd.read_csv('out/products.csv')

In [5]:
df1 = df[['Price 1', 'Curr1', 'Number Review 1', 'Review 1', 'Rating 1', 'ID', 'Name', 'Curr1 ID']]
df2 = df[['Price 2', 'Curr2', 'Number Review 2', 'Review 2', 'Rating 2', 'ID', 'Name', 'Curr2 ID']]
df3 = df[['Price 3', 'Curr3', 'Number Review 3', 'Review 3', 'Rating 3', 'ID', 'Name', 'Curr3 ID']]
df4 = df[['Price 4', 'Curr4', 'Number Review 4', 'Review 4', 'Rating 4', 'ID', 'Name', 'Curr4 ID']]
df5 = df[['Price 5', 'Curr5', 'Number Review 5', 'Review 5', 'Rating 5', 'ID', 'Name', 'Curr5 ID']]

In [6]:
df1 = df1.rename(columns={'Price 1' : 'Price', 'Curr1' : 'Curr', 'Number Review 1' : 'Number Review', \
                          'Review 1' : 'Review', 'Rating 1' : 'Rating', 'Curr1 ID' : 'CurID'})
df2 = df2.rename(columns={'Price 2' : 'Price', 'Curr2' : 'Curr', 'Number Review 2' : 'Number Review', \
                          'Review 2' : 'Review', 'Rating 2' : 'Rating', 'Curr2 ID' : 'CurID'})
df3 = df3.rename(columns={'Price 3' : 'Price', 'Curr3' : 'Curr', 'Number Review 3' : 'Number Review', \
                          'Review 3' : 'Review', 'Rating 3' : 'Rating', 'Curr3 ID' : 'CurID'})
df4 = df4.rename(columns={'Price 4' : 'Price', 'Curr4' : 'Curr', 'Number Review 4' : 'Number Review', \
                          'Review 4' : 'Review', 'Rating 4' : 'Rating', 'Curr4 ID' : 'CurID'})
df5 = df5.rename(columns={'Price 5' : 'Price', 'Curr5' : 'Curr', 'Number Review 5' : 'Number Review', \
                          'Review 5' : 'Review', 'Rating 5' : 'Rating', 'Curr5 ID' : 'CurID'})

In [7]:
newDf = pd.concat([df1, df2, df3, df4, df5])

In [8]:
newDf.rename(columns = {'Review':'review', 'Rating' : 'rating'}, inplace = True)

In [9]:
columns = ['id', 'createdDate', 'updatedDate', 'name', 'internalNote', 'isActive', 'authorId', 'statusId', \
          'productId', 'priceId', 'placeId', 'slug']

In [10]:
newDf = pd.concat([newDf, pd.DataFrame(columns = columns)])

In [11]:
import random
import time
 
 
def str_time_prop(start, end, time_format, prop):
    stime = time.mktime(time.strptime(start, time_format))
    etime = time.mktime(time.strptime(end, time_format))
    ptime = stime + prop * (etime - stime)
    return time.strftime(time_format, time.localtime(ptime))
 
 
def random_date(start, end, prop):
    return str_time_prop(start, end, '%Y-%m-%d', prop)

In [24]:
newDf['createdDate'] = [random_date("2021-1-1", "2022-4-30", random.random()) for i in range(1, newDf.shape[0] + 1)]
newDf['updatedDate'] = [i for i in newDf['createdDate']]
newDf['name'] = 'story'
newDf['internalNote'] = ''
newDf['isActive'] = True
newDf['statusId'] = 4
newDf['placeId'] = ''

In [23]:
def set_slug(value):
    return slugify('story' + str(value))

newDf['slug'] = newDf['id'].apply(set_slug)

In [14]:
def get_price_id(value):
    try:
        return priceDf[:][priceDf['ID'] == str(value)]['id'].values[0]
    except:
        return ''

newDf['priceId'] = newDf['CurID'].apply(get_price_id)

In [15]:
def get_product_id(value):
    try:
        return prodDf[:][prodDf['name'] == str(value)]['id'].values[0]
    except:
        return ''

newDf['productId'] = newDf['Name'].apply(get_product_id)

In [16]:
def get_author_id(value):
    try:
        temp = usersDf[:][(usersDf['Review1'] == str(value)) | \
         (usersDf['Review2'] == str(value)) | \
         (usersDf['Review3'] == str(value)) | \
         (usersDf['Review4'] == str(value)) | \
         (usersDf['Review6'] == str(value)) | \
         (usersDf['Review5'] == str(value))]['Username'].values[0].replace('\n', '')
        return users[:][users['username'].astype(str) == str(temp)]['id'].values[0]
    except:
        return ''

newDf['authorId'] = newDf['Number Review'].apply(get_author_id)

In [18]:
newDf = newDf.drop(['Price', 'Curr', 'Number Review', 'ID', 'Name'], axis=1)

In [19]:
newDf['rating'] = newDf['rating'].fillna(0)
newDf['id'] = [i for i in range(1, newDf.shape[0]+1)]

In [20]:
newDf.shape

(9715, 15)

In [21]:
newDf.to_csv('out/stories.csv', index=False, encoding='utf-8')

In [22]:
newDf.head()

,review,rating,CurID,id,createdDate,updatedDate,name,internalNote,isActive,authorId,statusId,productId,priceId,placeId,slug
0,In Tangkuban Perahu Volcano we 'must' have pai...,3.0,1_1,1,2021-05-17,2021-05-17,,,True,57,4,1,1,,in-tangkuban-perahu-volcano-we-must-have-paid-...
1,Not so many choices for ferry boats and they w...,1.0,2_1,2,2021-12-08,2021-12-08,,,True,427,4,2,2,,not-so-many-choices-for-ferry-boats-and-they-w...
2,I just love Bali! I've visited Bali many times...,5.0,3_1,3,2022-01-06,2022-01-06,,,True,57,4,3,3,,i-just-love-bali-i-ve-visited-bali-many-times-...
3,"I have paid the guide 450,000 rp per person fo...",4.0,4_1,4,2021-04-12,2021-04-12,,,True,235,4,4,4,,i-have-paid-the-guide-450000-rp-per-person-for...
4,NaN,0.0,5_1,5,2021-12-24,2021-12-24,,,True,960,4,5,5,,nan
